In [1]:
import sys
import os

current_dir = os.getcwd()  # 현재 작업 디렉토리
parent_dir = os.path.dirname(current_dir)  # 상위 디렉토리
DATA_PATH = os.path.join(parent_dir, 'data')
sys.path.append(parent_dir)

In [2]:
import os
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset


review_data_paths = glob.glob(os.path.join(DATA_PATH, 'review', '*.csv'))

# set cpu or cuda for default option
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device.type)


diner_df = pd.read_csv(os.path.join(DATA_PATH, "diner_df_20241118_yamyam.csv"))
review_df = pd.DataFrame()
for review_data_path in review_data_paths:
    review_df = pd.concat([review_df, pd.read_csv(review_data_path)], axis=0)


/tmp/ipykernel_1999534/2136300503.py:19: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  review_df = pd.concat([review_df, pd.read_csv(review_data_path)], axis=0)


In [3]:
import torch
from torch_geometric.data import HeteroData
from torch_geometric.nn import GraphSAGE
from torch_geometric.transforms import ToUndirected

# 필요한 컬럼 선택
review_df = review_df[['reviewer_id', 'diner_idx', 'reviewer_review_score', 'reviewer_avg', 'badge_level', 'reviewer_user_name']]
diner_df = diner_df[['diner_idx', 'diner_category_large', 'diner_category_middle', 
                     'diner_category_small', 'diner_review_avg', 'real_good_review_percent', 
                     'real_bad_review_percent']]


# 사용자 및 음식점 인덱스 생성
user_ids = sorted(list(review_df["reviewer_id"].unique()))
diner_ids = sorted(list(review_df["diner_idx"].unique()))

num_diners = len(diner_ids)
num_reviewers = len(user_ids)

user_id_map = {id: idx for idx, id in enumerate(user_ids)}
diner_id_map = {id: idx for idx, id in enumerate(diner_ids)}

# 사용자와 음식점 ID를 각각 숫자 인덱스로 변환
review_df['reviewer_id'] = review_df['reviewer_id'].map(user_id_map)
review_df['diner_idx'] = review_df['diner_idx'].map(diner_id_map)
diner_df['diner_idx'] = diner_df['diner_idx'].map(diner_id_map)

/home/elinha/.pyenv/versions/3.11.1/envs/gnn_env/lib/python3.11/site-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /home/elinha/.pyenv/versions/3.11.1/envs/gnn_env/lib/python3.11/site-packages/torch_spline_conv/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(


2. 그래프 생성
PyTorch Geometric의 HeteroData 객체를 사용하여 그래프를 구성합니다.

2.1. HeteroData 객체 초기화

In [4]:
diner_df[['diner_category_large', 'diner_category_middle', 'diner_category_small',
                           'diner_review_avg', 'real_good_review_percent', 'real_bad_review_percent']]

,diner_category_large,diner_category_middle,diner_category_small,diner_review_avg,real_good_review_percent,real_bad_review_percent
0,음식점,일식,일본식라면,4.7,33.333333,0.0
1,음식점,분식,NaN,3.4,33.333333,0.0
2,음식점,분식,NaN,4.6,33.333333,0.0
3,음식점,일식,일본식라면,4.7,33.333333,0.0
4,음식점,술집,일본식주점,4.7,33.333333,0.0
...,...,...,...,...,...,...
63548,음식점,한식,"육류,고기",4.0,40.000000,0.0
63549,음식점,한식,"육류,고기",4.0,40.000000,0.0
63550,음식점,한식,"육류,고기",4.0,40.000000,0.0
63551,음식점,한식,"육류,고기",4.0,40.000000,0.0


In [5]:
review_df

,reviewer_id,diner_idx,reviewer_review_score,reviewer_avg,badge_level,reviewer_user_name
0,162624,2393,5.0,5.0,4,아영
1,43469,2393,5.0,4.8,14,이봉 :)
2,202616,31401,5.0,4.3,6,이상희
3,133388,31401,5.0,5.0,12,nano
4,105084,31401,5.0,3.6,15,-
...,...,...,...,...,...,...
370396,36270,12359,5.0,4.0,6,A
370397,43846,4024,1.0,3.5,11,박동욱
370398,40414,4024,5.0,5.0,35,Jimmy
370399,138121,4024,4.0,4.0,29,잠원동크롱이


In [6]:
# PyG의 HeteroData 객체 생성
data = HeteroData()

# 노드 추가 (사용자와 음식점)
data['user'].num_nodes = len(user_ids)
data['restaurant'].num_nodes = len(diner_ids)


edge_index = torch.tensor([review_df['reviewer_id'].values, review_df['diner_idx'].values], dtype=torch.long)
data['user', 'interacts', 'restaurant'].edge_index = edge_index

# 엣지 추가 (리뷰 평점을 엣지 속성으로 사용)
data['user', 'interacts', 'restaurant'].edge_attr = torch.tensor(review_df['reviewer_review_score'].values, dtype=torch.float32)
data['restaurant', 'rev_interacts', 'user'].edge_index = edge_index[[1, 0]]  # 역방향 엣지

# 음식점 노드 특성 추가
# TODO: 
# # 카테고리 컬럼을 원-핫 인코딩
# diner_categories = pd.get_dummies(diner_df[['diner_category_middle', 'diner_category_small']])

# # 나머지 컬럼과 결합
# diner_features = pd.concat([diner_categories, diner_df[['diner_review_avg', 'real_good_review_percent', 'real_bad_review_percent']]], axis=1)

# # NaN 값을 0으로 대체 (또는 평균값으로 대체 가능)
# diner_features = diner_features.fillna(0).values

# # 텐서로 변환
# data['restaurant'].x = torch.tensor(diner_features, dtype=torch.float32)

# 'diner_category_middle' 원-핫 인코딩
diner_df['category_final'] = diner_df['diner_category_small'].fillna(diner_df['diner_category_middle'])

diner_category_middle_encoded = pd.get_dummies(diner_df['category_final'])

# 텐서로 변환하여 PyG의 그래프 데이터에 추가
diner_features = diner_category_middle_encoded.fillna(0).values
data['restaurant'].x = torch.tensor(diner_features, dtype=torch.float32)


# 사용자 노드 특성 추가: badge_level 및 reviewer_avg
user_features_df = review_df.groupby('reviewer_id').agg({
    'badge_level': 'first',  # 첫 번째 값 사용
    'reviewer_avg': 'first'  # 첫 번째 값 사용
}).reindex(user_ids).fillna(0)  # user_ids 순서로 재배열 및 NaN 채우기

# badge_level과 reviewer_avg를 합쳐서 노드 특성으로 변환
user_features = user_features_df.values
data['user'].x = torch.tensor(user_features, dtype=torch.float32)

# 그래프를 무방향으로 변환 (선택 사항)
data = ToUndirected()(data)


/home/elinha/.pyenv/versions/3.11.1/envs/gnn_env/lib/python3.11/site-packages/torch/utils/_device.py:79: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return func(*args, **kwargs)


3. 데이터 분할
train_test_split을 사용하여 훈련 및 테스트 세트로 분할합니다.

In [7]:
test_size=0.2  # 테스트 데이터 비율
min_reviews=3  # 최소 리뷰 수 (이보다 적은 리뷰를 가진 사용자는 제외)
random_state=42
stratify="reviewer_id"  # 리뷰어를 기준으로 층화 추출

# filter reviewer who wrote reviews more than min_reviews
reviewer2review_cnt = review_df["reviewer_id"].value_counts().to_dict()
reviewer_id_over = [reviewer_id for reviewer_id, cnt in reviewer2review_cnt.items() if cnt >= min_reviews]
review_over = review_df[lambda x: x["reviewer_id"].isin(reviewer_id_over)]
train, val = train_test_split(review_over,
                                test_size=test_size,
                                random_state=random_state,
                                stratify=review_over[stratify])

In [8]:
review_over

,reviewer_id,diner_idx,reviewer_review_score,reviewer_avg,badge_level,reviewer_user_name
0,162624,2393,5.0,5.0,4,아영
1,43469,2393,5.0,4.8,14,이봉 :)
3,133388,31401,5.0,5.0,12,nano
4,105084,31401,5.0,3.6,15,-
8,128222,15569,1.0,1.7,16,하얀나비
...,...,...,...,...,...,...
370394,222496,12359,5.0,4.4,56,마리오빠
370397,43846,4024,1.0,3.5,11,박동욱
370398,40414,4024,5.0,5.0,35,Jimmy
370399,138121,4024,4.0,4.0,29,잠원동크롱이


In [8]:
train_edge_index = torch.tensor([train['reviewer_id'].values, train['diner_idx'].values], dtype=torch.long)
test_edge_index = torch.tensor([val['reviewer_id'].values, val['diner_idx'].values], dtype=torch.long)
data['user', 'interacts', 'restaurant'].train_edge_index = train_edge_index
data['user', 'interacts', 'restaurant'].test_edge_index = test_edge_index

3. GraphSAGE 모델 정의<br>
3.1. 필요한 모듈 임포트

In [9]:
import torch.nn as nn
from torch_geometric.nn import SAGEConv, HeteroConv

3.2. 모델 클래스 정의

In [10]:
class GraphSAGERecommendationModel(nn.Module):
    def __init__(self, num_users, num_restaurants, embedding_dim, hidden_channels):
        super().__init__()
        
        # 사용자와 음식점 임베딩 레이어
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.restaurant_embedding = nn.Embedding(num_restaurants, embedding_dim)
        
        # GraphSAGE 레이어 정의
        self.conv1 = HeteroConv({
            ('user', 'interacts', 'restaurant'): SAGEConv((embedding_dim, embedding_dim), hidden_channels),
            ('restaurant', 'rev_interacts', 'user'): SAGEConv((embedding_dim, embedding_dim), hidden_channels),
        }, aggr='mean')
        
        self.conv2 = HeteroConv({
            ('user', 'interacts', 'restaurant'): SAGEConv((hidden_channels, hidden_channels), hidden_channels),
            ('restaurant', 'rev_interacts', 'user'): SAGEConv((hidden_channels, hidden_channels), hidden_channels),
        }, aggr='mean')
        
        # 최종 임베딩을 위한 선형 변환
        self.lin_user = nn.Linear(hidden_channels, embedding_dim)
        self.lin_restaurant = nn.Linear(hidden_channels, embedding_dim)
        
    def forward(self, data):
        x_dict = {
            'user': self.user_embedding.weight,
            'restaurant': self.restaurant_embedding.weight
        }
        edge_index_dict = data.edge_index_dict
        
        # 첫 번째 GraphSAGE 레이어
        x_dict = self.conv1(x_dict, edge_index_dict)
        
        # 두 번째 GraphSAGE 레이어
        x_dict = self.conv2(x_dict, edge_index_dict)
        
        # 최종 임베딩
        user_emb = self.lin_user(x_dict['user'])
        restaurant_emb = self.lin_restaurant(x_dict['restaurant'])
        
        return user_emb, restaurant_emb

# 모델 초기화
model = GraphSAGERecommendationModel(num_users=len(user_ids), num_restaurants=len(diner_ids), 
                                     embedding_dim=64, hidden_channels=128).to(device)


4. 모델 학습
4.1. 모델 및 옵티마이저 초기화

In [11]:
import torch
from torch.optim import Adam
import torch.nn as nn
import numpy as np

# 손실 함수와 옵티마이저 정의
criterion = nn.MSELoss()  # MSE 손실 함수
optimizer = Adam(model.parameters(), lr=0.01)

# Early Stopping 설정
patience = 10  # 성능 개선이 없을 경우 중단할 epoch 수
best_val_loss = float('inf')  # 초기 Best Loss
patience_counter = 0  # 성능 개선이 없는 epoch 수

# 학습 루프
num_epochs = 50  # 최대 에포크 수
for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    optimizer.zero_grad()  # 그래디언트 초기화

    # 모델의 예측 결과
    user_emb, restaurant_emb = model(data)  # 사용자와 음식점 임베딩 생성
    
    # 학습 데이터에서의 예측 값과 실제 값
    train_edge_index = data['user', 'interacts', 'restaurant'].edge_index
    y_true_train = data['user', 'interacts', 'restaurant'].edge_attr
    y_pred_train = (user_emb[train_edge_index[0]] * restaurant_emb[train_edge_index[1]]).sum(dim=1)
    
    # 학습 손실 계산
    train_loss = criterion(y_pred_train, y_true_train)
    train_loss.backward()
    optimizer.step()
    
    # 검증 데이터에서 성능 평가
    model.eval()
    with torch.no_grad():
        val_edge_index = data['user', 'interacts', 'restaurant'].test_edge_index
        y_true_val = val['reviewer_review_score'].values
        y_pred_val = (user_emb[val_edge_index[0]] * restaurant_emb[val_edge_index[1]]).sum(dim=1)
        val_loss = criterion(y_pred_val, torch.tensor(y_true_val, dtype=torch.float32, device=y_pred_val.device))
    
    # 출력 및 Early Stopping 체크
    print(f"Epoch {epoch + 1}, Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}")
    
    if val_loss.item() < best_val_loss:
        best_val_loss = val_loss.item()
        patience_counter = 0  # 성능이 개선되었으므로 초기화
        # 모델 저장
        best_model_state = model.state_dict()
    else:
        patience_counter += 1  # 성능 개선이 없으므로 카운터 증가
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

# 최적 모델 복원
model.load_state_dict(best_model_state)
print("Training completed with Early Stopping")


Epoch 1, Train Loss: 17.3553, Val Loss: 16.9846
Epoch 2, Train Loss: 9.6774, Val Loss: 8.2457
Epoch 3, Train Loss: 5.2051, Val Loss: 3.6287
Epoch 4, Train Loss: 10.1303, Val Loss: 10.0071
Epoch 5, Train Loss: 15.9915, Val Loss: 11.3583
Epoch 6, Train Loss: 7.4302, Val Loss: 7.5910
Epoch 7, Train Loss: 9.7450, Val Loss: 10.2903
Epoch 8, Train Loss: 7.7236, Val Loss: 8.4560
Epoch 9, Train Loss: 6.4888, Val Loss: 5.7648
Epoch 10, Train Loss: 6.2981, Val Loss: 4.4390
Epoch 11, Train Loss: 4.2957, Val Loss: 4.1008
Epoch 12, Train Loss: 4.5704, Val Loss: 4.6521
Epoch 13, Train Loss: 4.0173, Val Loss: 3.1000
Epoch 14, Train Loss: 3.9318, Val Loss: 2.8984
Epoch 15, Train Loss: 4.7043, Val Loss: 4.5698
Epoch 16, Train Loss: 3.2279, Val Loss: 2.6477
Epoch 17, Train Loss: 4.0732, Val Loss: 3.1188
Epoch 18, Train Loss: 3.9536, Val Loss: 4.0008
Epoch 19, Train Loss: 3.8617, Val Loss: 3.8397
Epoch 20, Train Loss: 3.0876, Val Loss: 2.5590
Epoch 21, Train Loss: 3.2442, Val Loss: 2.7111
Epoch 22, Train

4.2. 학습 데이터 준비

5.2. 모델 평가

In [12]:
# 평가 모드로 설정
model.eval()
with torch.no_grad():
    user_emb, restaurant_emb = model(data)
    
    val_edge_index = data['user', 'interacts', 'restaurant'].test_edge_index  # 평가 데이터 엣지
    y_true = val['reviewer_review_score'].values  # 실제 라벨로 'reviewer_review_score' 사용

    
    # 예측 값 계산
    y_pred = (user_emb[val_edge_index[0]] * restaurant_emb[val_edge_index[1]]).sum(dim=1)


In [13]:
y_pred_numpy = y_pred.detach().cpu().numpy()

In [14]:
from collections import Counter
Counter(y_true)

Counter({5.0: 61687, 4.0: 25454, 1.0: 17172, 3.0: 16244, 2.0: 8057})

In [15]:
import numpy as np


# Mean Average Precision (mAP) 계산
def mean_average_precision(y_true, y_pred):
    y_true_sorted = y_true[np.argsort(-y_pred)]
    cumsum = np.cumsum(y_true_sorted)
    precision_at_i = cumsum / (np.arange(1, len(y_true) + 1))
    return np.sum(precision_at_i * y_true_sorted) / np.sum(y_true_sorted)

# nDCG@K 계산
def ndcg_k(y_true, y_pred, k):
    idx = np.argsort(y_pred)[::-1][:k]
    y_true_k = np.take(y_true, idx)
    dcg = np.sum((2**y_true_k - 1) / np.log2(np.arange(2, k + 2)))
    idcg = np.sum((2**np.sort(y_true)[::-1][:k] - 1) / np.log2(np.arange(2, k + 2)))
    return dcg / idcg if idcg > 0 else 0

# 예시
map_score = mean_average_precision(y_true, y_pred_numpy)
print(f"mAP: {map_score:.4f}")

K = 30
ndcg_score = ndcg_k(y_true, y_pred_numpy, K)
print(f"nDCG@{K}: {ndcg_score:.4f}")


mAP: 4.3830
nDCG@30: 0.8460


추가 고려사항
데이터셋이 큰 경우:

메모리 부족 문제가 발생할 수 있습니다.
이 경우 미니배치 학습이나 NeighborSampler를 활용하여 샘플링 기반의 학습을 고려해 보세요.
노드 특징 추가:

사용자나 음식점에 대한 추가적인 특징(예: 프로필 정보, 카테고리 등)이 있다면 모델에 포함시켜 성능을 향상시킬 수 있습니다.
- 배지 레벨, 리뷰 쓴 수, 리뷰 쓴 업종, 
하이퍼파라미터 튜닝:

임베딩 차원, 학습률, 레이어 수 등을 변경하여 모델의 성능을 최적화할 수 있습니다.
모델 저장 및 로드:

학습된 모델을 저장하여 이후에 재사용할 수 있습니다.

In [16]:
# # 모델 저장
# torch.save(model.state_dict(), 'graphsage_model.pth')

# # 모델 로드
# model.load_state_dict(torch.load('graphsage_model.pth'))


5.3. 특정 사용자에게 추천 생성

In [17]:
# train과 val 데이터셋의 사용자 및 음식점 인덱스를 추출
train_user_indices = torch.tensor([uid for uid in train['reviewer_id'].values])
train_restaurant_indices = torch.tensor([diner_id for diner_id in train['diner_idx'].values])

val_user_indices = torch.tensor([uid for uid in val['reviewer_id'].values])
val_restaurant_indices = torch.tensor([diner_id for diner_id in val['diner_idx'].values])


In [18]:
review_df[review_df['reviewer_user_name'] == '로기']

,reviewer_id,diner_idx,reviewer_review_score,reviewer_avg,badge_level,reviewer_user_name
145663,33072,18090,5.0,4.2,4,로기
210420,89729,27277,2.0,3.6,21,로기
218513,89729,37954,3.0,3.6,21,로기
221235,89729,38985,5.0,3.6,21,로기
30436,148806,25028,5.0,5.0,35,로기
...,...,...,...,...,...,...
269356,148806,3949,5.0,5.0,35,로기
270667,148806,3443,5.0,5.0,35,로기
271456,148806,16353,5.0,5.0,35,로기
272736,148806,34679,5.0,5.0,35,로기


In [21]:
# 필요한 변수 설정
user_id = 893438059  # 특정 사용자 ID
user_idx = user_id_map[user_id]  # user_id를 인덱스로 변환

# 해당 사용자의 임베딩 벡터
user_vector = user_emb[user_idx]

# 모든 음식점 점수 계산
scores = (restaurant_emb @ user_vector).cpu().numpy()

# 이미 상호작용한 음식점 제외
interacted_restaurants = torch.cat([
    train_restaurant_indices[train_user_indices == user_idx],
    val_restaurant_indices[val_user_indices == user_idx]
]).cpu().numpy()

# 마스킹된 점수 생성
mask = np.ones(len(scores), dtype=bool)
mask[interacted_restaurants] = False
filtered_scores = scores[mask]  # 마스킹된 점수

# 원래 점수 배열의 인덱스를 마스킹 처리
original_indices = np.arange(len(scores))[mask]

# 상위 10개 음식점의 원래 인덱스 추출
top_k = 10
top_indices = original_indices[np.argsort(filtered_scores)[-top_k:][::-1]]

# 상위 점수와 ID 매핑
ranked_restaurants = [(idx, scores[idx]) for idx in top_indices]
inv_diner_mapping = {v: k for k, v in diner_id_map.items()}  # 인덱스 -> 원본 ID 역매핑

# 랭킹 결과 출력
print(f"Recommended restaurants for user {user_id}")
for rank, (restaurant_idx, score) in enumerate(ranked_restaurants, 1):
    print(f"Rank {rank}: https://place.map.kakao.com/{inv_diner_mapping[restaurant_idx]} (Score: {score:.4f})")


Recommended restaurants for user 893438059
Rank 1: https://place.map.kakao.com/1445391643 (Score: 8.2658)
Rank 2: https://place.map.kakao.com/1220697881 (Score: 7.9792)
Rank 3: https://place.map.kakao.com/1886557459 (Score: 7.4260)
Rank 4: https://place.map.kakao.com/1050744303 (Score: 6.9341)
Rank 5: https://place.map.kakao.com/231159704 (Score: 6.8114)
Rank 6: https://place.map.kakao.com/1365985075 (Score: 6.7622)
Rank 7: https://place.map.kakao.com/878807071 (Score: 6.7453)
Rank 8: https://place.map.kakao.com/9522592 (Score: 6.7395)
Rank 9: https://place.map.kakao.com/25448496 (Score: 6.7287)
Rank 10: https://place.map.kakao.com/23974088 (Score: 6.7051)
